####Подготовить данные с июня 2014 по май 2016:

In [0]:
!rm /content/ye*

rm: cannot remove '/content/ye*': No such file or directory


In [0]:
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
from scipy.stats import binned_statistic_2d
from datetime import timedelta
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
regions = pd.read_csv('/content/gdrive/My Drive/ml/c6/data/' + 'regions.csv', ';')

#вытащить уникальные границы из regions.csv
xbins = regions.west.unique()
xbins = np.append(xbins, -73.70001)
ybins = regions.south.unique()
ybins = np.append(ybins, 40.91553)

In [0]:
addr = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_"
# month_list = ['2014-04', '2014-05',
#               '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11',
#               '2014-12', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05',
#               '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11',
#               '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05',
#               '2014-06'
#              ]

month_list = ['2014-04', '2014-05']

In [0]:
def aggregator(date, xbins, ybins):
  
  #загрузка
  addr = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_"
  link = str(addr + date + ".csv")
  !wget $link
  file = str("/content/yellow_tripdata_" + date + ".csv")
  data = pd.read_csv(file)
  
  #предобработка
  data = data.rename(columns = {data.columns[1]: 'tpep_pickup_datetime'})
  data = data.rename(columns = {data.columns[2]: 'tpep_dropoff_datetime'})
  data = data.rename(columns = {data.columns[3]: 'passenger_count'})
  data = data.rename(columns = {data.columns[4]: 'trip_distance'})
  data = data.rename(columns = {data.columns[5]: 'pickup_longitude'})
  data = data.rename(columns = {data.columns[6]: 'pickup_latitude'})
  
  #фильтрация
  data.drop(data[data.tpep_pickup_datetime == data.tpep_dropoff_datetime].index , inplace=True)
  data.drop(data[data.passenger_count == 0].index , inplace=True)
  data.drop(data[data.trip_distance == 0].index , inplace=True)
  data.drop(data[data.pickup_longitude < -74.25559].index , inplace=True)
  data.drop(data[data.pickup_longitude > -73.70001].index , inplace=True)
  data.drop(data[data.pickup_latitude  <  40.49612].index , inplace=True)
  data.drop(data[data.pickup_latitude  >  40.91553].index , inplace=True)
  
  #конвертация
  data.tpep_pickup_datetime  = data.tpep_pickup_datetime.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  data.tpep_dropoff_datetime = data.tpep_dropoff_datetime.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  
  start_time = min(data.tpep_pickup_datetime)
  hour_list = []
  days = calendar.monthrange(start_time.year,start_time.month)[1]
  hours_cnt = days * 24

  #цикл для указанного количества дней в месяце
  for i in range(hours_cnt):
    current_hour = data[(data.tpep_pickup_datetime >= start_time + timedelta(hours=1) * (i)) & (data.tpep_pickup_datetime < start_time + timedelta(hours=1) * (i + 1))]
    arr = current_hour[['pickup_longitude', 'pickup_latitude']].values
    ret = binned_statistic_2d(arr[:,0],arr[:,1],None,'count',bins=[xbins, ybins])
    hour_string = [ret.statistic[x][y] for x in range(50) for y in range(50)]
    hour_list.append(hour_string)

  #создать dataframe из массива. В качестве индекса использовать дату и час.
  regions_list = ['region_' + str(x) for x in range(1,2500 + 1)]
  hour_index = [start_time + timedelta(hours=1) * (x) for x in range(hours_cnt)]
  regions_df = pd.DataFrame(hour_list, columns = regions_list, index = hour_index)

  #экспорт CSV
  name = str(start_time.year) + '-' + str(start_time.month) + '.csv'
  regions_df.to_csv(name)
  
  !rm /content/ye*
  
  return start_time, ' : DONE'

for i in range(len(month_list)):
  aggregator(month_list[i], xbins, ybins)